Last Update
12-11-2025

In [ ]:
#@title 💽 INSTALL RVC
from multiprocessing import cpu_count
cpu_cores = cpu_count()
post_process = False
LOGS_PATH = "/content/Applio/logs"
BACKUPS_PATH = "/content/drive/MyDrive/ApplioBackup"

%cd /content
!git config --global advice.detachedHead false
!git clone https://github.com/IAHispano/Applio --branch 3.5.1 --single-branch
%cd /content/Applio

# Install older python
!apt update -y
!apt install -y python3.11 python3.11-distutils python3.11-dev portaudio19-dev
!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 2
!update-alternatives --set python3 /usr/bin/python3.11
from sys import path
path.append('/usr/local/lib/python3.11/dist-packages')

print("Installing requirements...")
!curl -LsSf https://astral.sh/uv/install.sh | sh
!uv pip install -q -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu128 --index-strategy unsafe-best-match
!uv pip install -q jupyter-ui-poll
!python core.py "prerequisites" --models "True" --pretraineds_hifigan "True"
print("Finished installing requirements!")



### **Infer**



***اسم مدلی که باید در بخش نام مدل وارد کنید به یکی از حالتهای زیر است***

*   **یا اسم فایل زیپ مدل بدون پسوند زیپ است**

*   **pth یا اسم فایل اصلی مدل بدون پسوند**

*   **یا همان اسمی است که هنگام ساخت مدل تعیین کردید**






In [ ]:
#@title Load Model (ZIP, TAR.GZ, or Drive Link)
Model_Name = "" #@param {type:"string"}
MODEL_LINK = "" #@param {type:"string"}

import gdown
import os
import glob
import shutil
import tarfile  # Import tarfile for signature check

if not Model_Name:
    print("❌ Error: Please enter the Model Name.")
else:
    # Create model directory in target path
    target_dir = f"/content/Applio/logs/{Model_Name}"
    os.makedirs(target_dir, exist_ok=True)
    print(f"Target directory created/exists: {target_dir}")

    # --- Download ---
    downloaded_file_path = "/content/downloaded_model"
    try:
        if 'drive.google.com' in MODEL_LINK:
            print("Downloading from Google Drive...")
            gdown.download(MODEL_LINK, downloaded_file_path, quiet=False, fuzzy=True)
        elif MODEL_LINK.endswith(('.zip', '.tar.gz')):
            print(f"Downloading from direct link ({MODEL_LINK})...")
            os.system(f'wget -O {downloaded_file_path} "{MODEL_LINK}"')
        else:
            print("❌ Error: Unsupported link type. Please provide a Google Drive link or a direct link to a .zip or .tar.gz file.")
            downloaded_file_path = None  # Indicate download failure

        if downloaded_file_path and os.path.exists(downloaded_file_path):
            print("✅ Download successful.")
        elif downloaded_file_path:
            print("❌ Error: Download failed or file not found after download attempt.")
            downloaded_file_path = None

    except Exception as e:
        print(f"❌ Error during download: {e}")
        downloaded_file_path = None

    # --- Extraction ---
    if downloaded_file_path:
        file_type = None
        try:
            # Try identifying as tar.gz first
            with tarfile.open(downloaded_file_path, 'r:*') as tar:  # auto-detect compression
                file_type = 'tar'
                print("Detected file type: tar archive")
                temp_extract_dir = "/content/temp_extract"
                os.makedirs(temp_extract_dir, exist_ok=True)
                print(f"Extracting {downloaded_file_path} to {temp_extract_dir}...")
                tar.extractall(path=temp_extract_dir)

        except tarfile.ReadError:
            # If not a tarfile, try as zip
            try:
                import zipfile
                if zipfile.is_zipfile(downloaded_file_path):
                    file_type = 'zip'
                    print("Detected file type: zip archive")
                    print(f"Extracting {downloaded_file_path} to {target_dir}...")
                    with zipfile.ZipFile(downloaded_file_path, 'r') as zip_ref:
                        zip_ref.extractall(target_dir)
                else:
                    print("❌ Error: File is not a valid zip or tar archive.")
            except Exception as zip_e:
                print(f"❌ Error processing as zip file: {zip_e}")

        except Exception as e:
            print(f"❌ Error during extraction detection/setup: {e}")

        # --- Copying/Moving files specifically for TAR archives ---
        if file_type == 'tar':
            files_copied = False
            # Search and copy required files using os.walk
            for root, dirs, files in os.walk(temp_extract_dir):
                for file in files:
                    if file.endswith('.pth'):
                        source_path = os.path.join(root, file)
                        dest_path = os.path.join(target_dir, file)
                        shutil.copy2(source_path, dest_path)
                        print(f"Copied PTH: {file} to {target_dir}")
                        files_copied = True

                    if file.endswith('.index'):
                        source_path = os.path.join(root, file)
                        dest_path = os.path.join(target_dir, file)
                        shutil.copy2(source_path, dest_path)
                        print(f"Copied INDEX: {file} to {target_dir}")
                        files_copied = True

            if not files_copied:
                print("⚠️ Warning: No .pth or .index files found during deep search in extracted tar.")

            # Clean up temp extraction dir for tar
            if os.path.exists(temp_extract_dir):
                shutil.rmtree(temp_extract_dir)
                print(f"Removed temporary extraction directory: {temp_extract_dir}")

        # --- Cleanup Extraneous PTH Files (for BOTH zip and tar) ---
        print("\n🧹 Starting cleanup of extraneous PTH files...")
        expected_model_pth_filename = f"{Model_Name}.pth"
        expected_model_pth_full_path = os.path.join(target_dir, expected_model_pth_filename)

        # List all .pth files currently in the target directory
        pth_files_in_target = glob.glob(os.path.join(target_dir, "*.pth"))
        found_expected_pth = False

        if not pth_files_in_target:
            print(f"❌ Warning: No .pth files found in the final target directory {target_dir}.")
        else:
            print(f"Found PTH files: {[os.path.basename(f) for f in pth_files_in_target]}")
            # Check if the main expected file exists
            if expected_model_pth_full_path in pth_files_in_target:
                found_expected_pth = True
                print(f"✅ Expected model file '{expected_model_pth_filename}' found.")
            else:
                print(f"⚠️ Warning: Expected model file '{expected_model_pth_filename}' NOT found.")

            # Remove files starting with G_ or D_ OR any pth file if the expected one exists but this one isn't it
            for pth_file_path in pth_files_in_target:
                filename = os.path.basename(pth_file_path)
                if filename.startswith("G_") or filename.startswith("D_"):
                    try:
                        os.remove(pth_file_path)
                        print(f"🧹 Removed checkpoint file: {filename}")
                    except OSError as e:
                        print(f"❌ Error removing file {filename}: {e}")
                elif found_expected_pth and pth_file_path != expected_model_pth_full_path:
                    try:
                        os.remove(pth_file_path)
                        print(f"🧹 Removed other PTH file (since expected exists): {filename}")
                    except OSError as e:
                        print(f"❌ Error removing file {filename}: {e}")

        # 🔍 اینجا منطق پیشنهادی نام صحیح مدل را اضافه می‌کنیم
        final_pth_files = glob.glob(os.path.join(target_dir, "*.pth"))
        if final_pth_files:
            # اگر نام وارد شده باعث نشده فایل {Model_Name}.pth ساخته شود،
            # ولی یک یا چند pth دیگر هست، یکی را پیشنهاد بده
            if expected_model_pth_full_path not in final_pth_files:
                # ساده‌ترین حالت: از آخرین pth به‌عنوان نام صحیح استفاده کن
                suggested_full = sorted(final_pth_files)[-1]
                suggested_base = os.path.basename(suggested_full)
                suggested_name = os.path.splitext(suggested_base)[0]

                print("\n⚠️ توجه: به نظر می‌رسد نامی که برای مدل وارد کردید با نام واقعی فایل pth یکی نیست.")
                print(f"نام صحیح مدل بر اساس فایل موجود احتمالاً این است:")
                print(f"👉  {suggested_name}")
                print("لطفاً همین نام را بدون پسوند در فیلد نام مدل وارد کنید تا سلول اینفرِنس درست کار کند.")

        # Final listing to confirm
        print(f"\n✅ Extraction and cleanup complete for model '{Model_Name}'. Final contents:")
        os.system(f'ls -la "{target_dir}"')

        # Clean up the downloaded archive file
        os.remove(downloaded_file_path)
        print(f"Removed downloaded archive: {downloaded_file_path}")

    else:
        print("Skipping extraction due to download failure.")


In [ ]:

#@title 🎶 Upload Target Audio
from google.colab import files
import shutil
import os
import glob

print("⏳ لطفا فایل صوتی خود را آپلود کنید...")
uploaded = files.upload()

if uploaded:
    # پاک کردن هر فایل قبلی با نام input.*
    for f in glob.glob("/content/input.*"):
        os.remove(f)

    original_filename = list(uploaded.keys())[0]
    file_extension = os.path.splitext(original_filename)[1]
    new_filename = f"input{file_extension}"
    new_filepath = f"/content/{new_filename}"

    shutil.copy2(original_filename, new_filepath)

    print(f"✅ فایل صوتی با موفقیت در {new_filepath} ذخیره شد")
    print("\n📂 تأیید وجود فایل:")
    !ls -l "$new_filepath"

In [ ]:

#@title 🎶 Load Audio from Drive or Upload
#@markdown این سلول فایل صوتی رو از درایو دانلود می‌کنه

AUDIO_LINK = "" #@param {type:"string"}

import os
import mimetypes
from IPython.display import Audio, display, clear_output
import shutil

# استخراج File ID از لینک
file_id = AUDIO_LINK.split("/d/")[1].split("/")[0]
print("🎯 File ID:", file_id)

# دانلود با wget
!wget -q --no-check-certificate "https://docs.google.com/uc?export=download&id={file_id}" -O temp_audio

# تشخیص نوع فایل
mime_type, _ = mimetypes.guess_type("temp_audio")

if mime_type:
    if "wav" in mime_type:
        original_extension = ".wav"
    elif "mpeg" in mime_type or "mp3" in mime_type:
        original_extension = ".mp3"
    elif "ogg" in mime_type:
        original_extension = ".ogg"
    elif "flac" in mime_type:
        original_extension = ".flac"
    else:
        original_extension = ".wav"  # پیش‌فرض
else:
    original_extension = ".wav"

# تغییر نام
new_filename = f"input{original_extension}"
new_filepath = f"/content/{new_filename}"
shutil.move("temp_audio", new_filepath)

# نمایش و پخش
clear_output(wait=True)
display(Audio(new_filepath))
print(f"✅ فایل صوتی ذخیره شد در: {new_filepath}")
!ls -lh "$new_filepath"

In [ ]:
# @title Run Inference
# @markdown ####  `اسم مدل را وارد کن`
%cd /content/Applio
from pathlib import Path

model_name = ""  # @param {type:"string"}
model_path = Path(f"{LOGS_PATH}/{model_name}")
if not (model_path.exists() and model_path.is_dir()):
    raise FileNotFoundError(f"Model directory not found: {model_path.resolve()}")

# Select either the last checkpoint or the final weight
!ls -t "{model_path}"/"{model_name}"_*e_*s.pth "{model_path}"/"{model_name}".pth 2> /dev/null | head -n 1 > /tmp/pth.txt
pth_file = open("/tmp/pth.txt", "r").read().strip()
if pth_file == "":
  raise FileNotFoundError(f"No model weight found in directory: {model_path.resolve()}\nMake sure that the file is properly named (e.g. \"{model_name}.pth)\"")

!ls -t "{model_path}"/*.index | head -n 1 > /tmp/index.txt
index_file = open("/tmp/index.txt", "r").read().strip()

input_path = "/content/input.wav"
output_path = "/content/output.wav"
export_format = "WAV"  # @param ['WAV', 'MP3', 'FLAC', 'OGG', 'M4A'] {allow-input: false}
f0_method = "rmvpe"  # @param ["crepe", "crepe-tiny", "rmvpe", "fcpe", "hybrid[rmvpe+fcpe]"] {allow-input: false}
# @markdown ####  `تنظیم پیچ صدا`
f0_up_key = 0  # @param {type:"slider", min:-24, max:24, step:0}
rms_mix_rate = 0.8  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
protect = 0.5  # @param {type:"slider", min:0.0, max:0.5, step:0.1}
index_rate = 0.7  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
clean_strength = 0.7  # @param {type:"slider", min:0.0, max:1.0, step:0.1}
split_audio = False  # @param{type:"boolean"}
clean_audio = False  # @param{type:"boolean"}
f0_autotune = False  # @param{type:"boolean"}
formant_shift = False # @param{type:"boolean"}
formant_qfrency = 1.0 # @param {type:"slider", min:1.0, max:16.0, step:0.1}
formant_timbre = 1.0 # @param {type:"slider", min:1.0, max:16.0, step:0.1}
embedder_model = "contentvec" # @param ["contentvec", "chinese-hubert-base", "japanese-hubert-base", "korean-hubert-base", "custom"] {allow-input: false}
embedder_model_custom = "" # @param {type:"string"}

!rm -f "{output_path}"
if post_process:
  !python core.py infer --pitch "{f0_up_key}" --volume_envelope "{rms_mix_rate}" --index_rate "{index_rate}" --protect "{protect}" --f0_autotune "{f0_autotune}" --f0_method "{f0_method}" --input_path "{input_path}" --output_path "{output_path}" --pth_path "{pth_file}" --index_path "{index_file}" --split_audio "{split_audio}" --clean_audio "{clean_audio}" --clean_strength "{clean_strength}" --export_format "{export_format}" --embedder_model "{embedder_model}" --embedder_model_custom "{embedder_model_custom}" --formant_shifting "{formant_shift}" --formant_qfrency "{formant_qfrency}" --formant_timbre "{formant_timbre}" --post_process "{post_process}" --reverb "{reverb}" --pitch_shift "{pitch_shift}" --limiter "{limiter}" --gain "{gain}" --distortion "{distortion}" --chorus "{chorus}" --bitcrush "{bitcrush}" --clipping "{clipping}" --compressor "{compressor}" --delay "{delay}" --reverb_room_size "{reverb_room_size}" --reverb_damping "{reverb_damping}" --reverb_wet_gain "{reverb_wet_gain}" --reverb_dry_gain "{reverb_dry_gain}" --reverb_width "{reverb_width}" --reverb_freeze_mode "{reverb_freeze_mode}" --pitch_shift_semitones "{pitch_shift_semitones}" --limiter_threshold "{limiter_threshold}" --limiter_release_time "{limiter_release_time}" --gain_db "{gain_db}" --distortion_gain "{distortion_gain}" --chorus_rate "{chorus_rate}" --chorus_depth "{chorus_depth}" --chorus_center_delay "{chorus_center_delay}" --chorus_feedback "{chorus_feedback}" --chorus_mix "{chorus_mix}" --bitcrush_bit_depth "{bitcrush_bit_depth}" --clipping_threshold "{clipping_threshold}" --compressor_threshold "{compressor_threshold}" --compressor_ratio "{compressor_ratio}" --compressor_attack "{compressor_attack}" --compressor_release "{compressor_release}" --delay_seconds "{delay_seconds}" --delay_feedback "{delay_feedback}" --delay_mix "{delay_mix}"
else:
  !python core.py infer --pitch "{f0_up_key}" --volume_envelope "{rms_mix_rate}" --index_rate "{index_rate}" --protect "{protect}" --f0_autotune "{f0_autotune}" --f0_method "{f0_method}" --input_path "{input_path}" --output_path "{output_path}" --pth_path "{pth_file}" --index_path "{index_file}" --split_audio "{split_audio}" --clean_audio "{clean_audio}" --clean_strength "{clean_strength}" --export_format "{export_format}" --embedder_model "{embedder_model}" --embedder_model_custom "{embedder_model_custom}" --formant_shifting "{formant_shift}" --formant_qfrency "{formant_qfrency}" --formant_timbre "{formant_timbre}" --post_process "{post_process}"

if Path(output_path).exists():
  from IPython.display import Audio, display
  output_path = output_path.replace(".wav", f".{export_format.lower()}")
  display(Audio(output_path, autoplay=True))

In [ ]:

#@title ⬇️ Download Final Output
from google.colab import files
import glob
import os
from IPython.display import Audio, display, clear_output


output_files = glob.glob("/content/output.*")
if not output_files:
    print("هیچ فایل خروجی پیدا نشد!")
else:
    # جدیدترین بر اساس زمان ساخت
    latest_file = max(output_files, key=os.path.getmtime)
    filename = os.path.basename(latest_file)

    clear_output(wait=True)
    print(f"آخرین خروجی تولید شده:")
    print(f"🎵 {filename}")
    print(f"📏 حجم: {os.path.getsize(latest_file)/1024/1024:.2f} MB")
    print(f"⏰ زمان ساخت: {os.path.getmtime(latest_file):.0f}")

    # پخش صدا قبل از دانلود
    display(Audio(latest_file, autoplay=False))

    # دانلود خودکار
    files.download(latest_file)

    print("\nدانلود شروع شد...")